In [1]:
import numpy as np
import plotly.graph_objects as go

from multiprocess import Pool
from functools import partial

import games

import cpb_gaussian



def evaluate_parallel(nbCores, n_folds, horizon, alg, game, type):

    print("nbCores:", nbCores, "nbFolds:", n_folds, "Horizon:", horizon)
    pool = Pool(processes = nbCores) 
    task = Evaluation(horizon, type)

    np.random.seed(1)
    distributions = []

    for jobid in range(n_folds):
        
        if type == 'easy' :
            p = np.random.uniform(0, 0.2) if np.random.uniform(0,1)>0.5 else np.random.uniform(0.8, 1)
        else:
            p = np.random.uniform(0.4,0.6)
            
        distributions.append( [p, 1-p] )

    return np.asarray(  pool.map( partial( task.eval_policy_once, alg, game ), zip(distributions ,range(n_folds)) ) ) 

class Evaluation:

    def __init__(self, horizon, type ):
        self.horizon = horizon
        self.type = type
        # self.outcome_distribution = outcome_distribution

    def get_outcomes(self, game, job_id):
        # self.means = runif_in_simplex( self.game.n_outcomes )
        outcomes = np.random.choice( game.n_outcomes , p= list( game.outcome_dist.values() ), size= self.horizon) 
        return outcomes

    def get_feedback(self, game, action, outcome):
        return game.FeedbackMatrix[ action ][ outcome ]

    def eval_policy_once(self, alg, game, job):

        alg.reset()

        distribution, jobid = job

        #print('seed {} distribution {}'.format(jobid, distribution)) 
        np.random.seed(jobid)
        
        # outcome_distribution =  {'spam':0.05,'ham':0.95}

        outcome_distribution = {'spam':distribution[0],'ham':distribution[1]}

        # p = get_easy() if game.mode == 'easy' else get_harsch() 
        # outcome_distribution =  {'a':p[0],'b':p[1],'c':p[2],'d':p[3],'e':p[4]}

        game.set_outcome_distribution( outcome_distribution, jobid )
        optimal_action = game.i_star

        action_counter = np.zeros( (game.n_actions, self.horizon) )
        cum_regret = []

        # generate outcomes obliviously
        outcomes = self.get_outcomes(game, jobid)
        # outcomes, summary = self.distribution_shift(game )

        for t in range(self.horizon):

            # policy chooses one action
            action = alg.get_action(t)

            # Environment chooses one outcome
            outcome = outcomes[t]

            #print('t', t, 'action', action, 'outcome', outcome, )

            feedback =  self.get_feedback( game, action, outcome )

            alg.update(action, feedback, outcome, None, t)
            
            # print('nu', alg.nu / alg.n )

            for i in range(game.n_actions):
                if i == action:
                    action_counter[i][t] = action_counter[i][t-1] + 1
                else:
                    
                    action_counter[i][t] = action_counter[i][t-1]

        # regret = []
        # for t in range(self.horizon):
        #     regret.append(  self.delta_t( game, summary, t )  @ action_counter[:,t] )
            # cum_regret.append(  game.LossMatrix[action,outcome] - min( game.LossMatrix[...,outcome ] )  )
            
        regret = np.array( [ game.delta(i) for i in range(game.n_actions) ]).T @ action_counter
        
        return regret #np.cumsum(cum_regret)

In [ ]:
n_cores = 16
n_folds = 100
horizon = 100000

game = games.apple_tasting(False)

algos = [ cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, False, 1, 20, True),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, False, 1, 20, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, False, 1/8, 20, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, False, 1/16, 20, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, False, 1/32, 20, False)  ]

colors = [  [0,0,0], [250,0,0], [200,0,0], [150,0,0] , [100,0,0]   ] 
labels = [   'Uniform', 'Gaussian1',  'Gaussian18', 'Gaussian116','Gaussian132'   ] 

fig1 = go.Figure(    )
fig2 = go.Figure(    )


for alg, color, label in zip( algos, colors, labels):
    r,g,b = color
    result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'harsch')
    np.save('./results/benchmark_randcbp/apple_tasting/hard_{}_{}_{}'.format(horizon,n_folds, label), result)
    # result = np.load( './results/benchmark_randcbp/apple_tasting/hard_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    fig1.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b)), mode='lines',  name=label )) # 

    fig1.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=True ) )

    result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'easy')
    np.save('./results/benchmark_randcbp/apple_tasting/easy_{}_{}_{}'.format(horizon,n_folds, label), result)
    # result = np.load( './results/benchmark_randcbp/apple_tasting/easy_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    fig2.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b)), mode='lines',  name=label )) # 

    fig2.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=True ) )

fig1.update_yaxes(range=[0, 50] )
fig1.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.77), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                  font=dict(size=13,) )
fig1.show()

fig2.update_yaxes(range=[0, 10] )
fig2.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.77), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ) )
fig2.show()


In [ ]:
n_cores = 8
n_folds = 100
horizon = 2500


game = games.label_efficient(  )

algos = [ cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1, 20, True),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1, 20, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1/8, 20, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1/16, 20, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1/32, 20, False)  ]

colors = [  [0,0,0], [250,0,0], [200,0,0], [150,0,0] , [100,0,0]   ] 
labels = [   'Uniform', 'Gaussian1',  'Gaussian18', 'Gaussian116','Gaussian132'   ] 

fig1 = go.Figure(    )
fig2 = go.Figure(    )


for alg, color, label in zip( algos, colors, labels):
    r,g,b = color
    # result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'harsch')
    # np.save('./results/benchmark_randcbp/LE_hard_{}_{}_{}'.format(horizon,n_folds, label), result)
    result = np.load( './results/benchmark_randcbp/LE_hard_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    fig1.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b)), mode='lines',  name=label )) # 

    fig1.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False ) )

    # result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'easy')
    # np.save('./results/benchmark_randcbp/LE_easy_{}_{}_{}'.format(horizon,n_folds, label), result)
    result = np.load( './results/benchmark_randcbp/LE_easy_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    fig2.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b)), mode='lines',  name=label )) # 

    fig2.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=True ) )

fig1.update_yaxes(range=[0, 200] )
fig1.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.77), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                  font=dict(size=13,) )
fig1.show()

fig2.update_yaxes(range=[0, 30] )
fig2.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.77), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                  font=dict(size=13,) )
fig2.show()

In [ ]:
n_cores = 8
n_folds = 100
horizon = 2500

game = games.apple_tasting(False)
algos = [ cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1/16, 5, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1/16, 10, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1/16, 20, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1/16, 100, False)  ]

colors = [  [0,0,0], [250,0,0], [200,0,0], [150,0,0] , [100,0,0]   ] 
labels = [  'M5',  'M10', 'M20','M100'   ] 

fig1 = go.Figure(    )
fig2 = go.Figure(    )


for alg, color, label in zip( algos, colors, labels):
    r,g,b = color
    # result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'harsch')
    # np.save('./results/benchmark_randcbp/AT_hard_{}_{}_{}'.format(horizon,n_folds, label), result)
    result = np.load( './results/benchmark_randcbp/AT_hard_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    fig1.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b)), mode='lines',  name=label )) # 

    fig1.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False ) )

    # result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'easy')
    # np.save('./results/benchmark_randcbp/AT_easy_{}_{}_{}'.format(horizon,n_folds, label), result)
    result = np.load( './results/benchmark_randcbp/AT_easy_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    fig2.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b)), mode='lines',  name=label )) # 

    fig2.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False ) )

fig1.update_yaxes(range=[0, 50] )
fig1.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.77), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                  font=dict(size=13,) )
fig1.show()

fig2.update_yaxes(range=[0, 10] )
fig2.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.77), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                  font=dict(size=13,) )
fig2.show()


In [ ]:
n_cores = 8
n_folds = 100
horizon = 2500

fig1 = go.Figure(    )

result = np.load( './results/benchmark_randcbp/LE_hard_{}_{}_{}.npy'.format(horizon,n_folds, 'M5') )

fig1.add_trace(go.Scatter(x=xcoords, y=result[0], line=dict(color='rgb({},{},{})'.format(r,g,b), width = 0.7), mode='lines', name = 'M = 5' ) )
a = [ fig1.add_trace(go.Scatter(x=xcoords, y=result[i], line=dict(color='rgb({},{},{})'.format(r,g,b), width = 0.7), mode='lines',   showlegend=False) ) for i in range( len(result-1) ) ] # 

fig1.update_yaxes(range=[0, 3], type="log")
fig1.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.77), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Log-Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                  font=dict(size=13,) )
fig1.show()



fig2 = go.Figure(    )

result = np.load( './results/benchmark_randcbp/LE_hard_{}_{}_{}.npy'.format(horizon,n_folds, 'M10') )

fig2.add_trace(go.Scatter(x=xcoords, y=result[0], line=dict(color='rgb({},{},{})'.format(r,g,b), width = 0.7), mode='lines', name = 'M = 10' ) )
a = [ fig2.add_trace(go.Scatter(x=xcoords, y=result[i], line=dict(color='rgb({},{},{})'.format(r,g,b), width = 0.7), mode='lines',   showlegend=False) ) for i in range( len(result-1) ) ] # 

fig2.update_yaxes(range=[0, 3], type="log")
fig2.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.77), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Log-Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                  font=dict(size=13,) )
fig2.show()

In [ ]:
n_cores = 8
n_folds = 100
horizon = 2500

game = games.label_efficient()

algos = [ cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1/16, 5, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1/16, 10, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1/16, 20, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1/16, 100, False)  ]

colors = [  [0,0,0], [250,0,0], [200,0,0], [150,0,0] , [100,0,0]   ] 
labels = [  'M5',  'M10', 'M20','M100'   ] 

fig1 = go.Figure(    )
fig2 = go.Figure(    )


for alg, color, label in zip( algos, colors, labels):
    r,g,b = color
    # result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'harsch')
    # np.save('./results/benchmark_randcbp/LE_hard_{}_{}_{}'.format(horizon,n_folds, label), result)
    result = np.load( './results/benchmark_randcbp/LE_hard_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    fig1.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b)), mode='lines',  name=label )) # 

    fig1.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=True ) )

    # result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'easy')
    # np.save('./results/benchmark_randcbp/LE_easy_{}_{}_{}'.format(horizon,n_folds, label), result)
    result = np.load( './results/benchmark_randcbp/LE_easy_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    fig2.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b)), mode='lines',  name=label )) # 

    fig2.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=True ) )

fig1.update_yaxes(range=[0, 700] )
fig1.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.77), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                  font=dict(size=13,) )
fig1.show()

fig2.update_yaxes(range=[0, 50] )
fig2.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.77), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                  font=dict(size=13,) )
fig2.show()


In [ ]:
n_cores = 8
n_folds = 100
horizon = 2500

fig1 = go.Figure(    )

result = np.load( './results/benchmark_randcbp/LE_easy_{}_{}_{}.npy'.format(horizon,n_folds, 'M5') )

fig1.add_trace(go.Scatter(x=xcoords, y=result[0], line=dict(color='rgb({},{},{})'.format(r,g,b), width = 0.7), mode='lines', name = 'M = 5' ) )
a = [ fig1.add_trace(go.Scatter(x=xcoords, y=result[i], line=dict(color='rgb({},{},{})'.format(r,g,b), width = 0.7), mode='lines',   showlegend=False) ) for i in range( len(result-1) ) ] # 

fig1.update_yaxes(range=[0, 2.4], type="log" )
fig1.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.77), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Log-Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                  font=dict(size=13,) )
fig1.show()



fig2 = go.Figure(    )

result = np.load( './results/benchmark_randcbp/LE_easy_{}_{}_{}.npy'.format(horizon,n_folds, 'M10') )

fig2.add_trace(go.Scatter(x=xcoords, y=result[0], line=dict(color='rgb({},{},{})'.format(r,g,b), width = 0.7), mode='lines', name = 'M = 10' ) )
a = [ fig2.add_trace(go.Scatter(x=xcoords, y=result[i], line=dict(color='rgb({},{},{})'.format(r,g,b), width = 0.7), mode='lines',   showlegend=False) ) for i in range( len(result-1) ) ] # 


fig2.update_yaxes(range=[0, 2.4], type="log")
fig2.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.77), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Log-Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                  font=dict(size=13,) )
fig2.show()

In [ ]:
n_cores = 8
n_folds = 100
horizon = 2500

game = games.label_efficient()

algos = [ cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1/16, 5, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, False, 1/16, 5, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1/16, 10, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, False, 1/16, 10, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1/16, 20, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, False, 1/16, 20, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, True, 1/16, 100, False),
          cpb_gaussian.CPB_gaussian(  game, horizon, 1.01, False, 1/16, 100, False)  ]

colors = [   [250,0,0], [0,250,0], [200,0,0], [0,200,0], [150,0,0], [0,150,0], [100,0,0], [0,100,0]   ] 
labels = [  'M5f',   'M5f2',  'M10f',   'M10f2', 'M20f',   'M20f2', 'M100f',  'M100f2'   ] 

fig1 = go.Figure(    )
fig2 = go.Figure(    )

for alg, color, label in zip( algos, colors, labels):
    r,g,b = color
    # result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'harsch')
    # np.save('./results/benchmark_randcbp/LE_hard_{}_{}_{}'.format(horizon,n_folds, label), result)
    result = np.load( './results/benchmark_randcbp/LE_hard_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    fig1.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b)), mode='lines',  name=label )) # 

    fig1.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=True ) )

    # result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'easy')
    # np.save('./results/benchmark_randcbp/LE_easy_{}_{}_{}'.format(horizon,n_folds, label), result)
    result = np.load( './results/benchmark_randcbp/LE_easy_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    fig2.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b)), mode='lines',  name=label )) # 

    fig2.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=True ) )

fig1.update_yaxes(range=[0, 700] )
fig1.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.77), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                  font=dict(size=13,) )
fig1.show()

fig2.update_yaxes(range=[0, 50] )
fig2.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.77), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                  font=dict(size=13,) )
fig2.show()